In [ ]:
import pandas as pd 
import rioxarray as rio
import xarray as xr
import numpy as np
from tqdm.notebook import tqdm
df = pd.read_parquet('grid_1km.parquet',columns=['X_LLC','Y_LLC','TOT_P_2021'])


In [ ]:
df = pd.read_parquet('grid_1km.parquet')
df.iloc[0]

In [ ]:
crs = 'EPSG:3035'
yc = np.arange(df.Y_LLC.min(),df.Y_LLC.max()+1000,1000)+500
xc = np.arange(df.X_LLC.min(),df.X_LLC.max()+1000,1000)+500
df['col']=((df.X_LLC-df.X_LLC.min())/1000).astype(int)
df['row']=((df.Y_LLC-df.Y_LLC.min())/1000).astype(int)

dtype = np.int32

pop = np.zeros((len(yc),len(xc)), dtype=np.int32)-1

#for index,item in tqdm(df.iterrows(), total=len(df)):
#    pop[item.row,item.col] = item.TOT_P_2021
pop[df.row.values,df.col.values] = df.TOT_P_2021.values
np.max(pop.ravel()),np.max(df.TOT_P_2021.values)

In [ ]:

# Create a rioxarray from the numpy arrays
da = xr.DataArray(pop, dims=['y', 'x'])
da = da.assign_coords({'x': xc, 'y': yc})
da = da.rio.write_crs(crs)

da.rio.to_raster(raster_path="pop2021.tif", driver="COG", nodata=-1, predictor=3)
#da.rio.to_raster(raster_path="pop2021.tif", tiled=True, windowed=True, compress='LZW', predictor=2, nodata=-1)




In [ ]:
da